## 1- Definition of arguments for function usage

In [1]:
import sys
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from utils import *
import argparse
sys.argv = ['']

parser = argparse.ArgumentParser(description='Parameters training')
parser.add_argument('--model_architecture', type=str, default="VGG16", help='....')
parser.add_argument('--method', type=str, default="random", help='....')
parser.add_argument('--dataset', type=str, default="CIFAR10", help='....')
parser.add_argument('--batch_size', type=int, default=8, help='....')
parser.add_argument('--num_epochs', type=int, default=40, help='....')
parser.add_argument('--learning_rate', type=float, default=1e-3, help='....')
parser.add_argument('--optimizer_val', type=str, default="SGD", help='....')
parser.add_argument('--model_type', type=str, default="UNPRUNED", help='....')
parser.add_argument('--device', type=str, default=None, help='....')
parser.add_argument('--model_input', default=torch.ones((1, 3, 224, 224)), help='....')
parser.add_argument('--eval_metric', default="accuracy", help='....')
parser.add_argument('--pruning_seed', type=int, default=23, help='....')
parser.add_argument('--list_pruning', type=list, default = [0.6,0.6,0.53,0.53,0.4,0.4,0.4,0.5,0.5,0.5,0.6,0.6,0.6,0.5,0.5,0], help='....')

args = parser.parse_args()

if args.device is None:
    import torch
    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Modify for training
#args.model_architecture = "ResNet18"
#args.num_epochs = 40
args.dataset = "Date_Fruit_7classes"
args.eval_metric = "f1_score"
custom_split = 1    # 1: if dataset is already divided into train and test folders, 
                    # 0: if all the images of the dataset are in a single folder



## Get Model, DATASET and TRAIN

In [3]:
train_loader, test_loader, num_classes = get_dataset(args, custom_split = custom_split)

In [4]:
model = get_model(num_classes, args)

In [5]:
train_model(train_loader = train_loader,
            test_loader = test_loader,
            model = model,
            num_classes = num_classes,
            args = args)

Epoch: [1/40]	 || Training Loss: 1.052	 || Val Loss: 0.480	 || Training F1-score: 0.599 	 ||  Val F1-score: 0.770 	 || Time: 00:02:01
Model Name: VGG16_Date_Fruit_7classes_UNPRUNED
Epoch: [2/40]	 || Training Loss: 0.514	 || Val Loss: 0.553	 || Training F1-score: 0.804 	 ||  Val F1-score: 0.748 	 || Time: 00:04:07
Epoch: [3/40]	 || Training Loss: 0.350	 || Val Loss: 0.516	 || Training F1-score: 0.875 	 ||  Val F1-score: 0.773 	 || Time: 00:06:19
Model Name: VGG16_Date_Fruit_7classes_UNPRUNED
Epoch: [4/40]	 || Training Loss: 0.194	 || Val Loss: 0.361	 || Training F1-score: 0.927 	 ||  Val F1-score: 0.827 	 || Time: 00:08:35
Model Name: VGG16_Date_Fruit_7classes_UNPRUNED
Epoch: [5/40]	 || Training Loss: 0.133	 || Val Loss: 0.362	 || Training F1-score: 0.953 	 ||  Val F1-score: 0.841 	 || Time: 00:10:53
Model Name: VGG16_Date_Fruit_7classes_UNPRUNED
Epoch: [6/40]	 || Training Loss: 0.105	 || Val Loss: 0.387	 || Training F1-score: 0.967 	 ||  Val F1-score: 0.839 	 || Time: 00:13:11
Epoch: [

## Pruning with multiple seeds

In [6]:
# Modify for pruning
dict_distri = {"HOMOGENEA" : [0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0],
                "CRECIENTE" : [0.15,0.15,0.2,0.2,0.3,0.3,0.3,0.43,0.43,0.44,0.45,0.45,0.46,0.3,0.3,0],
                "DECRECIENTE" : [0.45,0.45,0.35,0.35,0.3,0.3,0.3,0.25,0.25,0.25,0.16,0.16,0.13,0.3,0.3,0],
                "MENOS_MAS_MENOS" : [0.15,0.15,0.3,0.3,0.45,0.45,0.45,0.28,0.28,0.27,0.2,0.19,0.15,0.3,0.3,0],
                "MAS_MENOS_MAS" : [0.4,0.4,0.2,0.2,0.3,0.3,0.3,0.3,0.3,0.3,0.37,0.37,0.36,0.3,0.3,0]}

base_percentage = 30

In [7]:
list_seeds = [23,42,97,112,167]

for distri in dict_distri:
    args.list_pruning = dict_distri[distri]
    type_pruning = distri
    original_model_name = f'{args.model_architecture}_{args.dataset}_UNPRUNED'
    
    for seed in list_seeds:
        #load original model
        model = torch.load(f'models/{args.dataset}/{original_model_name}.pth')
        args.seed = seed
        args.model_type = f'{type_pruning}_{base_percentage}_PRUNED_SEED_{seed}'
        #prune original model
        prune_model(model, args)
        args.model_type = f'{type_pruning}_{base_percentage}_PRUNED_FT_SEED_{seed}'
        #retraining pruned model
        train_model(train_loader = train_loader,
                    test_loader = test_loader,
                    model = model,
                    num_classes = num_classes,
                    args = args)
        print('============================')

Epoch: [1/40]	 || Training Loss: 0.991	 || Val Loss: 0.543	 || Training F1-score: 0.637 	 ||  Val F1-score: 0.728 	 || Time: 00:01:28
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_23
Epoch: [2/40]	 || Training Loss: 0.533	 || Val Loss: 0.540	 || Training F1-score: 0.804 	 ||  Val F1-score: 0.724 	 || Time: 00:02:58
Epoch: [3/40]	 || Training Loss: 0.354	 || Val Loss: 0.425	 || Training F1-score: 0.878 	 ||  Val F1-score: 0.781 	 || Time: 00:04:26
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_23
Epoch: [4/40]	 || Training Loss: 0.273	 || Val Loss: 0.422	 || Training F1-score: 0.901 	 ||  Val F1-score: 0.814 	 || Time: 00:05:56
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_23
Epoch: [5/40]	 || Training Loss: 0.168	 || Val Loss: 0.839	 || Training F1-score: 0.943 	 ||  Val F1-score: 0.760 	 || Time: 00:07:27
Epoch: [6/40]	 || Training Loss: 0.150	 || Val Loss: 0.784	 || Training F1-score: 0.949 	 ||  Val F1-s

Epoch: [14/40]	 || Training Loss: 0.052	 || Val Loss: 0.399	 || Training F1-score: 0.985 	 ||  Val F1-score: 0.839 	 || Time: 00:22:38
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_42
Epoch: [15/40]	 || Training Loss: 0.024	 || Val Loss: 0.439	 || Training F1-score: 0.993 	 ||  Val F1-score: 0.840 	 || Time: 00:24:25
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_42
Epoch: [16/40]	 || Training Loss: 0.029	 || Val Loss: 0.352	 || Training F1-score: 0.991 	 ||  Val F1-score: 0.858 	 || Time: 00:26:08
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_42
Epoch: [17/40]	 || Training Loss: 0.015	 || Val Loss: 0.545	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.825 	 || Time: 00:27:48
Epoch: [18/40]	 || Training Loss: 0.036	 || Val Loss: 0.443	 || Training F1-score: 0.990 	 ||  Val F1-score: 0.832 	 || Time: 00:29:37
Epoch: [19/40]	 || Training Loss: 0.024	 || Val Loss: 0.532	 || Training F1-score: 0.992 	 ||  Va

Epoch: [28/40]	 || Training Loss: 0.020	 || Val Loss: 0.672	 || Training F1-score: 0.993 	 ||  Val F1-score: 0.805 	 || Time: 00:49:07
Epoch: [29/40]	 || Training Loss: 0.015	 || Val Loss: 0.513	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.818 	 || Time: 00:50:42
Epoch: [30/40]	 || Training Loss: 0.025	 || Val Loss: 0.903	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.784 	 || Time: 00:52:17
Epoch: [31/40]	 || Training Loss: 0.035	 || Val Loss: 0.532	 || Training F1-score: 0.990 	 ||  Val F1-score: 0.828 	 || Time: 00:53:53
Epoch: [32/40]	 || Training Loss: 0.025	 || Val Loss: 0.429	 || Training F1-score: 0.990 	 ||  Val F1-score: 0.838 	 || Time: 00:55:28
Epoch: [33/40]	 || Training Loss: 0.017	 || Val Loss: 0.576	 || Training F1-score: 0.994 	 ||  Val F1-score: 0.816 	 || Time: 00:57:04
Epoch: [34/40]	 || Training Loss: 0.007	 || Val Loss: 0.547	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.821 	 || Time: 00:58:40
Epoch: [35/40]	 || Training Loss: 0.018	 || Val Loss: 0

Epoch: [4/40]	 || Training Loss: 0.245	 || Val Loss: 0.651	 || Training F1-score: 0.908 	 ||  Val F1-score: 0.770 	 || Time: 00:06:39
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_167
Epoch: [5/40]	 || Training Loss: 0.198	 || Val Loss: 0.427	 || Training F1-score: 0.935 	 ||  Val F1-score: 0.808 	 || Time: 00:08:19
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_167
Epoch: [6/40]	 || Training Loss: 0.127	 || Val Loss: 0.425	 || Training F1-score: 0.956 	 ||  Val F1-score: 0.823 	 || Time: 00:09:57
Model Name: VGG16_Date_Fruit_7classes_random_HOMOGENEA_30_PRUNED_FT_SEED_167
Epoch: [7/40]	 || Training Loss: 0.109	 || Val Loss: 0.630	 || Training F1-score: 0.963 	 ||  Val F1-score: 0.799 	 || Time: 00:11:35
Epoch: [8/40]	 || Training Loss: 0.104	 || Val Loss: 0.609	 || Training F1-score: 0.964 	 ||  Val F1-score: 0.774 	 || Time: 00:13:11
Epoch: [9/40]	 || Training Loss: 0.075	 || Val Loss: 0.708	 || Training F1-score: 0.976 	 ||  Val F

Epoch: [19/40]	 || Training Loss: 0.036	 || Val Loss: 0.390	 || Training F1-score: 0.991 	 ||  Val F1-score: 0.847 	 || Time: 00:31:11
Model Name: VGG16_Date_Fruit_7classes_random_CRECIENTE_30_PRUNED_FT_SEED_23
Epoch: [20/40]	 || Training Loss: 0.068	 || Val Loss: 0.621	 || Training F1-score: 0.975 	 ||  Val F1-score: 0.783 	 || Time: 00:32:50
Epoch: [21/40]	 || Training Loss: 0.031	 || Val Loss: 0.530	 || Training F1-score: 0.990 	 ||  Val F1-score: 0.830 	 || Time: 00:34:26
Epoch: [22/40]	 || Training Loss: 0.027	 || Val Loss: 0.557	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.825 	 || Time: 00:36:09
Epoch: [23/40]	 || Training Loss: 0.029	 || Val Loss: 0.649	 || Training F1-score: 0.991 	 ||  Val F1-score: 0.795 	 || Time: 00:38:08
Epoch: [24/40]	 || Training Loss: 0.054	 || Val Loss: 0.689	 || Training F1-score: 0.982 	 ||  Val F1-score: 0.804 	 || Time: 00:39:43
Epoch: [25/40]	 || Training Loss: 0.016	 || Val Loss: 0.607	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.810

...

Epoch: [9/40]	 || Training Loss: 0.102	 || Val Loss: 0.536	 || Training F1-score: 0.965 	 ||  Val F1-score: 0.780 	 || Time: 00:14:05
Epoch: [10/40]	 || Training Loss: 0.100	 || Val Loss: 0.390	 || Training F1-score: 0.966 	 ||  Val F1-score: 0.833 	 || Time: 00:15:38
Model Name: VGG16_Date_Fruit_7classes_random_MENOS_MAS_MENOS_30_PRUNED_FT_SEED_167
Epoch: [11/40]	 || Training Loss: 0.067	 || Val Loss: 0.521	 || Training F1-score: 0.977 	 ||  Val F1-score: 0.817 	 || Time: 00:17:13
Epoch: [12/40]	 || Training Loss: 0.045	 || Val Loss: 0.918	 || Training F1-score: 0.988 	 ||  Val F1-score: 0.765 	 || Time: 00:18:45
Epoch: [13/40]	 || Training Loss: 0.057	 || Val Loss: 0.783	 || Training F1-score: 0.979 	 ||  Val F1-score: 0.763 	 || Time: 00:20:18
Epoch: [14/40]	 || Training Loss: 0.045	 || Val Loss: 0.606	 || Training F1-score: 0.986 	 ||  Val F1-score: 0.806 	 || Time: 00:21:50
Epoch: [15/40]	 || Training Loss: 0.048	 || Val Loss: 0.531	 || Training F1-score: 0.984 	 ||  Val F1-score:

Epoch: [24/40]	 || Training Loss: 0.016	 || Val Loss: 0.668	 || Training F1-score: 0.994 	 ||  Val F1-score: 0.815 	 || Time: 00:38:19
Epoch: [25/40]	 || Training Loss: 0.065	 || Val Loss: 0.585	 || Training F1-score: 0.982 	 ||  Val F1-score: 0.806 	 || Time: 00:39:56
Epoch: [26/40]	 || Training Loss: 0.013	 || Val Loss: 0.544	 || Training F1-score: 0.997 	 ||  Val F1-score: 0.830 	 || Time: 00:41:32
Epoch: [27/40]	 || Training Loss: 0.011	 || Val Loss: 0.584	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.821 	 || Time: 00:43:07
Epoch: [28/40]	 || Training Loss: 0.013	 || Val Loss: 0.692	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.805 	 || Time: 00:44:42
Epoch: [29/40]	 || Training Loss: 0.012	 || Val Loss: 0.572	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.817 	 || Time: 00:46:17
Epoch: [30/40]	 || Training Loss: 0.023	 || Val Loss: 0.563	 || Training F1-score: 0.992 	 ||  Val F1-score: 0.821 	 || Time: 00:47:52
Epoch: [31/40]	 || Training Loss: 0.018	 || Val Loss: 1

Epoch: [40/40]	 || Training Loss: 0.010	 || Val Loss: 0.531	 || Training F1-score: 0.997 	 ||  Val F1-score: 0.842 	 || Time: 01:03:50
Epoch: [1/40]	 || Training Loss: 0.927	 || Val Loss: 0.782	 || Training F1-score: 0.651 	 ||  Val F1-score: 0.704 	 || Time: 00:01:35
Model Name: VGG16_Date_Fruit_7classes_random_MAS_MENOS_MAS_30_PRUNED_FT_SEED_97
Epoch: [2/40]	 || Training Loss: 0.539	 || Val Loss: 0.662	 || Training F1-score: 0.799 	 ||  Val F1-score: 0.711 	 || Time: 00:03:12
Model Name: VGG16_Date_Fruit_7classes_random_MAS_MENOS_MAS_30_PRUNED_FT_SEED_97
Epoch: [3/40]	 || Training Loss: 0.379	 || Val Loss: 0.506	 || Training F1-score: 0.866 	 ||  Val F1-score: 0.760 	 || Time: 00:04:50
Model Name: VGG16_Date_Fruit_7classes_random_MAS_MENOS_MAS_30_PRUNED_FT_SEED_97
Epoch: [4/40]	 || Training Loss: 0.250	 || Val Loss: 0.423	 || Training F1-score: 0.915 	 ||  Val F1-score: 0.810 	 || Time: 00:06:27
Model Name: VGG16_Date_Fruit_7classes_random_MAS_MENOS_MAS_30_PRUNED_FT_SEED_97
Epoch: [5

Epoch: [13/40]	 || Training Loss: 0.046	 || Val Loss: 0.468	 || Training F1-score: 0.985 	 ||  Val F1-score: 0.822 	 || Time: 00:20:49
Epoch: [14/40]	 || Training Loss: 0.016	 || Val Loss: 0.620	 || Training F1-score: 0.995 	 ||  Val F1-score: 0.805 	 || Time: 00:22:25
Epoch: [15/40]	 || Training Loss: 0.030	 || Val Loss: 0.721	 || Training F1-score: 0.989 	 ||  Val F1-score: 0.794 	 || Time: 00:24:07
Epoch: [16/40]	 || Training Loss: 0.041	 || Val Loss: 0.728	 || Training F1-score: 0.988 	 ||  Val F1-score: 0.795 	 || Time: 00:25:54
Epoch: [17/40]	 || Training Loss: 0.054	 || Val Loss: 0.410	 || Training F1-score: 0.983 	 ||  Val F1-score: 0.848 	 || Time: 00:27:40
Model Name: VGG16_Date_Fruit_7classes_random_MAS_MENOS_MAS_30_PRUNED_FT_SEED_112
Epoch: [18/40]	 || Training Loss: 0.050	 || Val Loss: 0.575	 || Training F1-score: 0.985 	 ||  Val F1-score: 0.823 	 || Time: 00:29:18
Epoch: [19/40]	 || Training Loss: 0.023	 || Val Loss: 0.446	 || Training F1-score: 0.990 	 ||  Val F1-score: 

Epoch: [30/40]	 || Training Loss: 0.010	 || Val Loss: 0.582	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.821 	 || Time: 00:49:47
Epoch: [31/40]	 || Training Loss: 0.007	 || Val Loss: 0.517	 || Training F1-score: 0.998 	 ||  Val F1-score: 0.830 	 || Time: 00:51:24
Epoch: [32/40]	 || Training Loss: 0.035	 || Val Loss: 0.911	 || Training F1-score: 0.989 	 ||  Val F1-score: 0.761 	 || Time: 00:52:59
Epoch: [33/40]	 || Training Loss: 0.018	 || Val Loss: 0.523	 || Training F1-score: 0.993 	 ||  Val F1-score: 0.820 	 || Time: 00:54:35
Epoch: [34/40]	 || Training Loss: 0.022	 || Val Loss: 0.669	 || Training F1-score: 0.993 	 ||  Val F1-score: 0.804 	 || Time: 00:56:10
Epoch: [35/40]	 || Training Loss: 0.013	 || Val Loss: 0.457	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.841 	 || Time: 00:57:52
Epoch: [36/40]	 || Training Loss: 0.013	 || Val Loss: 0.532	 || Training F1-score: 0.996 	 ||  Val F1-score: 0.824 	 || Time: 00:59:39
Epoch: [37/40]	 || Training Loss: 0.012	 || Val Loss: 0